In [1]:
import pandas as pd

from dataset import load_data
from players_df import create_players_df, filter_d1_participants, add_test_drules_to_players_df
from blocks_df import create_blocks_df, merge_blocks_dataframes, analyze_blocks_dataframe, add_generalized_drule_column

In [2]:
d1 = load_data('../data/real-complete_only-v3.json')
d2 = load_data('../data/real-complete_only-v4.json')
d3 = load_data('../data/real-complete_only-v5_share_score.json')
d4 = load_data('../data/real-complete_only-v6_rule_share.json')
d5 = load_data('../data/real-complete_only-v8_partner_pred.json')
d6 = load_data('../data/real-complete_only-v10_full_info.json')

In [4]:
d2['00503b68-97a8-4307-a66b-ca4dd8b1b5cf-p195'].keys()

dict_keys(['relevantFeatures', 'withdraw_data', 'starttime', 'recruitment_info', 'relevantFeaturesBadValues', 'points', 'bonus_points', 'browser_fingerprint', 'route_order', 'questionnaireQuestions', 'user_data', 'partner_found', 'trial_num', 'browser_data', 'quizQuestions', 'withdraw', 'seedID', 'partner_left', 'totalDuration', 'demographic_form', 'conditions', 'consented', 'task_data', 'smile_config', 'partner_task_data', 'route_times', 'switch_to_solo', 'endtime', 'recruitment_service', 'done'])

In [23]:
id = '00503b68-97a8-4307-a66b-ca4dd8b1b5cf-p195'
phase = 2

# Print data for first 10 trials
for trial in range(20):
    print(f"\n--- Trial {trial} ---")
    trial_data = d2[id]['task_data'][phase][trial]
    print(f"response: {trial_data['response']}")
    print(f"approachOutcome: {trial_data['approachOutcome']}")
    print(f"pointChange: {trial_data['pointChange']}")


--- Trial 0 ---
response: approach
approachOutcome: 1
pointChange: 1

--- Trial 1 ---
response: approach
approachOutcome: 1
pointChange: 1

--- Trial 2 ---
response: avoid
approachOutcome: -1
pointChange: 0

--- Trial 3 ---
response: approach
approachOutcome: 1
pointChange: 1

--- Trial 4 ---
response: approach
approachOutcome: 1
pointChange: 1

--- Trial 5 ---
response: approach
approachOutcome: 1
pointChange: 1

--- Trial 6 ---
response: avoid
approachOutcome: -1
pointChange: 0

--- Trial 7 ---
response: approach
approachOutcome: 1
pointChange: 1

--- Trial 8 ---
response: approach
approachOutcome: 1
pointChange: 1

--- Trial 9 ---
response: approach
approachOutcome: 1
pointChange: 1

--- Trial 10 ---
response: approach
approachOutcome: 1
pointChange: 1

--- Trial 11 ---
response: avoid
approachOutcome: -1
pointChange: 0

--- Trial 12 ---
response: approach
approachOutcome: 1
pointChange: 1

--- Trial 13 ---
response: approach
approachOutcome: 1
pointChange: 1

--- Trial 14 ---
resp

In [ ]:
d2['00503b68-97a8-4307-a66b-ca4dd8b1b5cf-p195']['task_data'][0][7]

In [5]:
# For d1, only completed participants
players_df_d1 = create_players_df(d1, version='d1')
players_df_d1 = filter_d1_participants(players_df_d1)

# For d2, with specific whitelist (example from original function)
d2_whitelist = [
    '576ad50bc8a90000010d99c8',
    '61267e282a445af9eb95beae',
    '6331b9456956d880a1e4632f'
    # Add other whitelisted IDs as needed
]
players_df_d2 = create_players_df(d2, version='d2', pids_whitelist=d2_whitelist)

# For d3 and d4, only completed participants
players_df_d3 = create_players_df(d3, version='d3')
players_df_d4 = create_players_df(d4, version='d4')
players_df_d5 = create_players_df(d5, version='d5') 
players_df_d6 = create_players_df(d6, version='d6')

In [6]:
print("\nCreating blocks dataframes for each dataset...")

# Get ids for each dataset from individual players_df
d1_ids = players_df_d1['id'].tolist()
d2_ids = players_df_d2['id'].tolist()
d3_ids = players_df_d3['id'].tolist()
d4_ids = players_df_d4['id'].tolist()
d5_ids = players_df_d5['id'].tolist()
d6_ids = players_df_d6['id'].tolist()

# Create blocks_df for each dataset using the filtered ids
epsilon = 1 # 0.25
blocks_d1 = create_blocks_df(d1, ids=d1_ids, n_phases=4, phase_blocks=[4,2,4,2], epsilon=epsilon)
blocks_d2 = create_blocks_df(d2, ids=d2_ids, n_phases=4, phase_blocks=[4,2,4,2], epsilon=epsilon) 
blocks_d3 = create_blocks_df(d3, ids=d3_ids, n_phases=4, phase_blocks=[4,2,4,2], epsilon=epsilon)
blocks_d4 = create_blocks_df(d4, ids=d4_ids, n_phases=4, phase_blocks=[4,2,4,2], epsilon=epsilon)
blocks_d5 = create_blocks_df(d5, ids=d5_ids, n_phases=5, phase_blocks=[4,2,4,4,2], epsilon=epsilon) # , partner_prediction_phase=2)
blocks_d6 = create_blocks_df(d6, ids=d6_ids, n_phases=4, phase_blocks=[4,2,4,2], epsilon=epsilon)


Creating blocks dataframes for each dataset...


/Users/rab/dyadic-learning-trap-paper/data_processing/blocks_df.py:110: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  new_df.loc[:, 'drule'] = pd.Categorical(new_df['drule'], categories=['2d', '1d_a', '1d_b', 'neither'])
/Users/rab/dyadic-learning-trap-paper/data_processing/blocks_df.py:110: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  new_df.loc[:, 'drule'] = pd.Categorical(new_df['drule'], categories=['2d', '1d_a', '1d_b', 'neither'])
/Users/rab/dyadic-learning-trap-paper/data_processing/blocks_df.py:110: FutureWarning: In a future version, `df.il

In [7]:
# Merge all blocks dataframes using the new function
blocks_all = merge_blocks_dataframes(
    [blocks_d1, blocks_d2, blocks_d3, blocks_d4, blocks_d5, blocks_d6],
    dataset_labels=['1', '2', '3', '4', '5', '6']
)

# Analyze the merged dataframe
analyze_blocks_dataframe(blocks_all)

# Add generalized drule column
blocks_all = add_generalized_drule_column(blocks_all)


Merged blocks dataframe size: 13976

Checking for NaN values in blocks_all...
Number of unique IDs with NaN values: 197

Columns with NaN values:
  points: 788 NaN values

Breakdown of IDs with NaNs by dataset:
  1: 0 IDs with NaNs
  2: 0 IDs with NaNs
  3: 0 IDs with NaNs
  4: 0 IDs with NaNs
  5: 197 IDs with NaNs
  6: 0 IDs with NaNs


In [8]:
# Add test drules to each players_df separately (before merging)
print("Adding test drules to individual datasets...")

# For d1-d4 and d6: standard phases (first_test_phase=1, second_test_phase=3)
players_df_d1 = add_test_drules_to_players_df(players_df_d1, blocks_d1, epsilon=2, first_test_phase=1, second_test_phase=3)
players_df_d2 = add_test_drules_to_players_df(players_df_d2, blocks_d2, epsilon=2, first_test_phase=1, second_test_phase=3) 
players_df_d3 = add_test_drules_to_players_df(players_df_d3, blocks_d3, epsilon=2, first_test_phase=1, second_test_phase=3)
players_df_d4 = add_test_drules_to_players_df(players_df_d4, blocks_d4, epsilon=2, first_test_phase=1, second_test_phase=3)
players_df_d6 = add_test_drules_to_players_df(players_df_d6, blocks_d6, epsilon=2, first_test_phase=1, second_test_phase=3)

# For d5: different phase structure (first_test_phase=1, second_test_phase=4)
players_df_d5 = add_test_drules_to_players_df(players_df_d5, blocks_d5, epsilon=2, first_test_phase=1, second_test_phase=4)

print("Test drules added to all datasets")

Adding test drules to individual datasets...
Test drules added to all datasets


In [9]:
# Merge players_df

# Add dataset identifier to each dataframe
players_df_d1['dataset'] = '1'
players_df_d2['dataset'] = '2'
players_df_d3['dataset'] = '3'
players_df_d4['dataset'] = '4'
players_df_d5['dataset'] = '5'
players_df_d6['dataset'] = '6'

# Combine all dataframes and fill NaN with None
players_df_all = pd.concat([players_df_d1, players_df_d2, players_df_d3, players_df_d4, players_df_d5, players_df_d6], axis=0, ignore_index=True)
players_df_all = players_df_all.where(pd.notnull(players_df_all), None)

# Sort columns for better readability, keeping dataset first
cols = ['dataset'] + sorted([col for col in players_df_all.columns if col != 'dataset'])
players_df_all = players_df_all[cols]

In [10]:
blocks_all

,phase,block,id,short_id,game_type,points,drule,drule_error_2d,drule_error_1d_a,drule_error_1d_b,drule_error_1d,dataset,drule_gen
0,0,0,001d5ec2-0ca4-468a-b230-9efaca6b945f-p83,001-p83,solo,-7.0,neither,7.0,11.0,7.0,7.0,1,neither
1,0,1,001d5ec2-0ca4-468a-b230-9efaca6b945f-p83,001-p83,solo,2.0,neither,6.0,10.0,4.0,4.0,1,neither
2,0,2,001d5ec2-0ca4-468a-b230-9efaca6b945f-p83,001-p83,solo,8.0,1d_b,4.0,8.0,0.0,0.0,1,1d
3,0,3,001d5ec2-0ca4-468a-b230-9efaca6b945f-p83,001-p83,solo,8.0,1d_b,4.0,8.0,0.0,0.0,1,1d
4,1,0,001d5ec2-0ca4-468a-b230-9efaca6b945f-p83,001-p83,solo,8.0,1d_b,4.0,8.0,0.0,0.0,1,1d
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13971,2,1,ffd957cd-7b97-47c1-bd81-dce5b90c2534-p25,ffd-p25,solo,3.0,neither,5.0,5.0,5.0,5.0,6,neither
13972,2,2,ffd957cd-7b97-47c1-bd81-dce5b90c2534-p25,ffd-p25,solo,7.0,neither,5.0,7.0,5.0,5.0,6,neither
13973,2,3,ffd957cd-7b97-47c1-bd81-dce5b90c2534-p25,ffd-p25,solo,4.0,neither,8.0,8.0,6.0,6.0,6,neither
13974,3,0,ffd957cd-7b97-47c1-bd81-dce5b90c2534-p25,ffd-p25,solo,6.0,neither,6.0,8.0,2.0,2.0,6,neither


In [11]:
players_df_all

,dataset,bonus_points,external_aid,first_half_points,first_test_drule,first_test_drule_gen,first_test_points,game_type,id,partner_id,...,partner_rule,partner_short_id,points,prolific_id,second_half_points,second_test_drule,second_test_drule_gen,second_test_points,short_id,time_on_share_page
0,1,NaN,False,47,1d_b,1d,16,solo,001d5ec2-0ca4-468a-b230-9efaca6b945f-p83,adf34386-97d8-41fa-b1de-d0c9195891af-p68,...,None,adf-p68,95,5dd31760fb7d41000c6610e3,48,1d_b,1d,16,001-p83,NaN
1,1,NaN,False,42,2d,2d,24,duo,0125b293-7c4d-46b1-8f98-3cf91d30e47f-p99,b586b9a4-c6a8-432b-a135-ce17d319d5ba-p100,...,None,b58-p100,114,5e387f2c0d70ef099d5e5a4f,72,2d,2d,24,012-p99,NaN
2,1,NaN,False,76,2d,2d,24,solo,02b493f1-589e-44ba-9af0-91f4d8c5a29d-p328,832eab28-7844-461b-a18a-8bff19e7fa9c-p327,...,None,832-p327,148,640d43c9451ce148162a7643,72,2d,2d,24,02b-p328,NaN
3,1,NaN,False,17,neither,neither,11,duo,02c9cdaf-d2c4-49a2-b715-16e3834f1ae1-p316,5c93f901-ccd2-4339-ba9d-cd75df9aeead-p320,...,None,5c9-p320,40,63d81bc7c218d6206ae6689b,23,1d_b,1d,11,02c-p316,NaN
4,1,NaN,False,39,1d_b,1d,11,duo,071cdd29-b5ae-457a-a3d4-ec6486cb4c81-p315,6d9cff58-a0d4-4cdf-9201-dc57f49b1e0e-p314,...,None,6d9-p314,83,56210effed6e5a000ac7f3a5,44,1d_b,1d,16,071-p315,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,6,0.0,False,50,2d,2d,24,solo,f46b5d22-a527-499a-a2e4-b0301ddd9a2b-p54,None,...,None,None,121,673204494167687dfb6a5747,71,2d,2d,23,f46-p54,NaN
1095,6,0.0,False,47,neither,neither,16,solo,f579071d-4014-4636-b24e-7bd52e6709d0-p15,None,...,None,None,94,663ff3e539df6ec6f060f875,47,2d,2d,22,f57-p15,NaN
1096,6,0.0,False,45,1d_b,1d,12,solo,f5fb3537-805b-45ed-ae19-142cb7f9795d-p98,None,...,None,None,93,65654bb65a81489784483bf0,48,1d_b,1d,16,f5f-p98,NaN
1097,6,0.0,False,57,1d_b,1d,16,solo,fc955006-2531-4c44-9b17-5bce8157a705-p26,None,...,None,None,115,672c4e78eeaae1dfe925f5e2,58,2d,2d,23,fc9-p26,NaN


In [14]:
players_df_all[players_df_all['external_aid'] == True]

,dataset,bonus_points,external_aid,first_half_points,first_test_drule,first_test_drule_gen,first_test_points,game_type,id,partner_id,...,partner_rule,partner_short_id,points,prolific_id,second_half_points,second_test_drule,second_test_drule_gen,second_test_points,short_id,time_on_share_page
6,1,NaN,True,49,1d_a,1d,16,solo,086d19a1-e4c6-43ab-bfd0-08d92d825c27-p143,6ed5654c-1cfb-4807-8ab3-9cab153891b4-p146,...,None,6ed-p146,97,60fbed5d2532f7768e7b5a29,48,1d_a,1d,16,086-p143,NaN
38,1,NaN,True,43,1d_b,1d,16,duo,2eba1a58-5db7-48b8-ba52-e9cf123aacc7-p127,3b1f19bd-e595-44e0-bf24-a1f3977eb490-p125,...,None,3b1-p125,91,58d876b14240e50001190090,48,1d_b,1d,16,2eb-p127,NaN
102,1,NaN,True,4,neither,neither,0,duo,82209325-5b9b-48a6-8122-ee27efabc8bd-p283,ab436631-37b8-4621-a3fc-247eaaf47093-p285,...,None,ab4-p285,27,63d3f987619fb984bbfd9693,23,neither,neither,8,822-p283,NaN
123,1,NaN,True,41,1d_b,1d,16,duo,968ff598-b543-40bb-9f28-9123d5b4cc7a-p202,c5ee247e-0952-4472-aaca-7891ba958612-p206,...,None,c5e-p206,89,5c1c137b0739430001693cf5,48,1d_b,1d,16,968-p202,NaN
174,1,NaN,True,55,neither,neither,15,duo,d222b9ff-934f-491c-9be4-a2f17059d1ef-p244,e097a217-0bda-41c7-8aba-c35bbc3f9f6f-p245,...,None,e09-p245,105,5c37db5fbef9a20001e3aee7,50,1d_a,1d,16,d22-p244,NaN
182,1,NaN,True,33,2d,2d,24,solo,db07315a-6b0d-456d-ba9e-5536f12f956c-p114,872197de-08b2-4af9-a503-05455e107dbc-p122,...,None,872-p122,105,5a84f454ae9a0b0001a9e4e5,72,2d,2d,24,db0-p114,NaN
199,1,NaN,True,55,2d,2d,24,duo,f268ce1d-b7f9-400a-8b28-6ce5374e729f-p7,3485a1cd-5bfa-4ad1-ba6b-f728bb527001-p8,...,None,348-p8,125,5e5410f5be932e06384a03ec,70,2d,2d,24,f26-p7,NaN
276,2,12.0,True,15,neither,neither,9,duo,5eef9b80-9b74-4b59-8709-cc5ca9d553c9-p192,None,...,2d,None,68,55bd8669fdf99b5bfc7d4cfc,53,2d,2d,24,5ee-p192,NaN
359,2,12.0,True,52,2d,2d,24,duo,c3076467-f462-4afd-8376-de45aa448d60-p109,None,...,2d,None,124,656dde1807d2e89595eaa3b6,72,2d,2d,24,c30-p109,NaN
364,2,12.0,True,20,1d_b,1d,16,duo,c9b9c16a-8adf-4121-9af8-fe1378e55547-p174,None,...,1d_a,None,68,6101c5bffd584fdda7def135,48,1d_b,1d,16,c9b-p174,NaN
